# Design & Deployment

## Exercises in Algorithm Design

```{admonition} Issues to Consider
- How many queries are required, and what kind of composition can we use?
  - Is parallel composition possible?
  - Should we use sequential composition, advanced composition, or a variant of differential privacy?
- Can we use the sparse vector technique?
- Can we use the exponential mechanism?
- How should we distribute the privacy budget?
- If there are unbounded sensitivities, how can we bound them?
- Would synthetic data help?
- Would post-processing to "de-noise" help?
```

## 1. Generalized Sample and Aggregate

Design a variant of sample and aggregate which does *not* require the analyst to specify the output range of the query function $f$.

```{tip}
Use SVT to find good upper and lower bounds on $f(x)$ for the whole dataset first. The result of $clip(f(x), lower, upper)$ has bounded sensitivity, so we can use this query with SVT. Then use sample and aggregate with these upper and lower bounds.
```



In [23]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

adult = pd.read_csv("adult_with_pii.csv")

#svt implementation
def above_threshold(queries, df, T, epsilon,on_fail="random"):
    T_hat = T + np.random.laplace(loc=0, scale = 2/epsilon)
    for idx, q in enumerate(queries):
        nu_i = np.random.laplace(loc=0, scale = 4/epsilon)
        if q(df) + nu_i >= T_hat:
            return idx

    # Handle failure case
    if on_fail == "random":
        print("random int returned")
        # if the algorithm "fails", return a random index
        # more convenient in certain use cases
        return random.randint(0, len(queries) - 1)
    else:
        return None
#our query
def age_sum_query(df):
    return df['Age'].sum()
# code from the book to compute the upper bound, and their respective counterparts for the lower bound
def create_query(query, b):
    return lambda df: age_sum_query(df) - b

def create_query_lower(query, b):
    return lambda df: b - age_sum_query(df)

bs = range(1750000,900000,-14)
bs_lower = range(900000,1750000, 14)
queries = [create_query(age_sum_query(adult),b) for b in bs]
queries_lower = [create_query_lower(age_sum_query(adult),b) for b in bs_lower]
epsilon = .1

upper_sum = bs[above_threshold(queries, adult, 0, epsilon)]
lower_sum = bs_lower[above_threshold(queries_lower, adult, 0, epsilon)]
print("lower bound: ", lower_sum , "\nupper bound: ", upper_sum , "\nactual value: " , age_sum_query(adult))


lower bound:  1360166 
upper bound:  1360324 
actual value:  1360238


In [19]:
def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity/epsilon)

# saa_age_sum is based on saa_avg_age from chapter 7
def saa_age_sum(k, epsilon, logging=False):
    df = adult

    # Calculate the number of rows in each chunk
    chunk_size = int(np.ceil(df.shape[0] / k))

    if logging:
        print(f'Chunk size: {chunk_size}')

    # Step 1: split `df` into chunks
    xs      = [df.iloc[i:i+chunk_size] for i in range(0,df.shape[0],chunk_size)]

    # Step 2: run f on each x_i and clip its output
    answers = [age_sum_query(x_i) for x_i in xs]

    u = upper_sum / k
    l = lower_sum / k
    clipped_answers = np.clip(answers, l, u)

    # Step 3: take the noisy sum of the clipped answers
    noisy_sum = laplace_mech(np.sum(clipped_answers), (u-l), epsilon)
    return noisy_sum

saa_age_sum(600, 1, logging=True)
#epsilon = 1,2 (0,1 x 2 for upper_sum and lower_sum berechnung + 1 for saa)

Chunk size: 55


np.float64(1344329.2259279098)

## 2. Summary Statistics

Design an algorithm to produce differentially private versions of the following statistics:

- Mean: $\mu = \frac{1}{n} \sum_{i=1}^n x_i$
- Variance: $var = \frac{1}{n} \sum_{i=1}^n (x_i - \mu)^2$
- Standard deviation: $\sigma = \sqrt{\frac{1}{n} \sum_{i=1}^n (x_i - \mu)^2}$

**Ideas**:

**Mean**

1. Use SVT to find upper and lower clipping bounds
2. Compute noisy sum and count, and derive mean by post-processing

**Variance**

1. Split it into a count query ($\frac{1}{n}$ - we have the answer from above) and a sum query
2. What's the sensitivity of $\sum_{i=1}^n (x_i - \mu)^2$? It's $b^2$; we can clip and compute $\sum_{i=1}^n (x_i - \mu)^2$, then multiply by (1) by post processing

**Standard Deviation**

1. Just take the square root of variance

Total queries:
- Lower clipping bound (SVT)
- Upper clipping bound (SVT)
- Noisy sum (mean)
- Noisy count
- Noisy sum (variance)

In [24]:
#mean
#compute upper bound via SVT

def age_sum_query(df, b):
    return df['Age'].clip(lower=0, upper=b).sum()

def create_query(b):
    return lambda df: age_sum_query(df, b) - age_sum_query(df, b + 1)

bs = range(1,150,2)
queries = [create_query(b) for b in bs]

#compute lower bound via SVT

def age_sum_query_lower(df, b):
    return df['Age'].clip(lower=b, upper=150).sum()

def create_query_lower(b):
    return lambda df: age_sum_query_lower(df, b) - age_sum_query_lower(df, b - 1)

bs_lower = range(150,1,-2)
queries_lower = [create_query_lower(b) for b in bs]
epsilon = .1
upper_age = bs[above_threshold(queries, adult, 0, epsilon)]
lower_age = bs[above_threshold(queries_lower, adult, 0, epsilon)]
print("lower bound: ", lower_age, " upper bound: ", upper_age)

lower bound:  3  upper bound:  95


In [26]:
def dp_mean_variance_stddef(epsilon):
    df = adult["Age"]

    answers = df.values

    u = upper_age
    l = lower_age
    clipped_answers = np.clip(answers, l, u)

    noisy_sum = laplace_mech(clipped_answers.sum(), (u-l), epsilon)
    noisy_count = laplace_mech(df.shape[0], 1, epsilon)
    one_div_n = 1 / noisy_count #needed later
    noisy_mean = one_div_n * noisy_sum

    var_sensitivity = max((l-noisy_mean)**2,(u-noisy_mean)**2)
    squared_diff = (clipped_answers - noisy_mean) ** 2
    noisy_sum_variance = laplace_mech(squared_diff.sum(), var_sensitivity, epsilon)
    noisy_variance = one_div_n * noisy_sum_variance
    noisy_stddev = np.sqrt(noisy_variance)

    return noisy_mean, noisy_variance, noisy_stddev

mean, variance, stddev = dp_mean_variance_stddef(1)
print("mean: ", mean, "\nvariance: ", variance, "\nstandard deviation: ", stddev)

mean:  41.77131595929292 
variance:  320.78817140332126 
standard deviation:  17.910560331919303


## 3. Heavy Hitters

Google's RAPPOR system {cite}`rappor` is designed to find the most popular settings for Chrome's home page. Design an algorithm which:

- Given a list of the 10,000 most popular web pages by traffic,
- Determines the top 10 most-popular home pages out of the 10,000 most popular web pages


```{tip}
Use parallel composition and take the noisy top 10
```

```{note}
RAPPOR operates in the **local** model of differential privacy, which will be introduced later on. Feel free to skip this exercise for now and return later!
```

**Lösung**

Ich interpretiere, dass ich Zugang zu den Nutzern habe und abfragen kann, ob Website x Ihre Homepage ist.

Ich interpretiere auch, dass diese Info locally dp zurückkommt.

Laut ChatGPT verwendet RAPPOR einen Mechanismus pro User Report (essentiell für die privacy Cost weiter unten)

10k mal Fragen, ob die Website die Homepage ist, würde 10000 * epsilon kosten

Also erstelle ich einen 10000-bit langen Bitstring, die Nutzer sollen den Bit auf 1 setzen, der ihre Homepage ist (kann auch keine der 10k sein)

Privacy cost = epsilon, da ein mal mit dem Nutzer interagiert wurde.

Die einzelnen Reports sind dank LDP schon noisy , also summiere ich die 1er Bits pro Website und gebe die 10 größten Summen zurück


## 4. Hierarchical Queries

Design an algorithm to produce summary statistics for the U.S. Census. Your algorithm should produce total population counts at the following levels:

- Census tract
- City / town
- ZIP Code
- County
- State
- USA

```{tip}

Idea 1: *Only* compute the bottom level (census tract), using parallel composition. Add up all the tract counts to get the city counts, and so on up the hierarchy. Advantage: lowers privacy budget.

Idea 2: Compute counts at all levels, using parallel composition for each level. Tune the budget split using real data; probably we need more accuracy for the smaller levels of the hierarchy.

Idea 3: As (2), but also use post-processing to re-scale lower levels of the hierarchy based on higher ones; truncate counts to whole numbers; move negative counts to 0.

```

**Lösung**

Idee 1: Die parallel composition wird hier angewendet, in dem man ein counting query parallel auf alle möglichen census tracts laufen lässt -> privacy cost = epsilon

Die Sensitivität von counting queries ist 1, also ist der noisy count gleich q(D) + Laplace(1/epsilon)

Die oberen Ebenen sind dann trivial, einfach die entsprechenden noisy counts zusammenfügen. - keine Erhöhung der privacy cost, da das unter post processing fällt.

Idee 2: Wir behalten die counting querys und die Berechnung des Noisy count bei, diesmal aber auf jeder Ebene.

Die Aufgabe ändert sich insofern, dass wir hier auch mit sequential composition arbeiten, da jeder Datensatz einmal pro Ebene vorkommt. Dieses Gesamt-Epsilon müssen wir sinnvoll aufteilen, die unteren Ebenen brauchen mehr privacy.

Mein Ansatz für die Aufteilung von epsilon wäre wiefolgt. Wir finden heraus, wieviele Möglichkeiten es pro Ebene (1 USA, 50 States, usw). Wir summieren diese Möglichkeiten (M) und rechnen dann mit M(Ebene) / M(Gesamt) den Anteil der Ebene an epsilon_gesamt aus.

 Da ich laut Angabe 'real data' verwenden soll, hier die Counts und die dazugehörigen Anteile an epsilon_gesamt für die USA:

| Ebene  | M  | %  |
|---|---|---|
| USA  | 1  | 0,00067  |
| States  | 50  | 0,03336  |
| Countys  | 3244  | 2,16454  |
| ZIP Codes  | 41552  | 27,72536  |
| Cities / Towns  | 19495  | 13,00794  |
| Census tracts  | 85528  | 57,06813  |
| Total  | 149870  | 100  |

Diese Aufteilung ist heuristisch und rechnerisch leicht nachvollziehbar. Dennoch würde ich noch die Prozent von ZIP Codes und Cities tauschen, da es ja nicht direkt darum geht, wie viele mögliche Werte eine Ebene hat, sondern wie fein die Ebene ist.
Je größer epsilon, desto größeren Einfluss hat das Rauschen (hier durch Laplace) auf den Count. Das ist bei der Ebene USA nicht schlimm, aber bei ZIP-Codes zb. schon. Es gibt zwar weniger Städte als Postleitzahlen, aber Städte sind dennoch die feinere Schicht.

Idee 3: Erweiterung von Idee 2. Alles passiert im post processing, der privacy cost erhöht sich also nicht. Wir skalieren die Daten, da die Summen der Ebenen nicht übereinstimmen.
Wir berechnen Count(USA) / Summe(Counts(States)) um den Skalierungsparameter herauszufinden.
Wir rechnen Skalierungsparameter * Count(State) für alle States. Nun gilt Summe(Counts(States)) = Count(USA).
Nun ist States die obere Schicht und Countys die untere. So gehen wir vor, bis wir die unterste Schicht erreicht haben und alle Schichten denselben Count ergeben.
Das entfernen von negativen Einträgen sowie das Runden geschieht zum Schluss, da wir es durch die Skalierung sonst mehrmals anwenden müssten.

## 5. Workloads of Range Queries

Design an algorithm to accurately answer a workload of *range queries*. Range queries are queries on a single table of the form: "how many rows have a value that is between $a$ and $b$?" (i.e. the count of rows which lie in a specific range). 

### Part 1
The whole workload is pre-specified as a finite sequence of ranges: $\{(a_1, b_1), \dots, (a_k, b_k)\}$, and 

### Part 2
The length of the workload $k$ is pre-specified, but queries arrive in a streaming fashion and must be answered as they arrive.

### Part 3
The workload may be infinite.

**Ideas**:

Just run each query with sequential composition.

For part 1, combine them so we can use $L2$ sensitivity. When $k$ is large, this will work well with Gaussian noise.

Or, build synthetic data:

- For each range $(i, i+1)$, find a count (parallel composition). This is a synthetic data representation! We can answer infinitely many queries by adding up the counts of all the segments in this histogram which are contained in the desired interval.
- For part 2, use SVT

For SVT: for each query in the stream, ask how far the real answer is from the synthetic data answer. If it's far, query the real answer's range (as a histogram, using parallel composition) and update the synthetic data. Otherwise just give the synthetic data answer. This way you *ONLY* pay for updates to the synthetic data.

**Lösung**

Jedes Query hintereinander zu beantworten wäre die naive Implementation, aber mit hoher privacy cost von k * epsilon (gilt für parts 1+2, bei Part 3 wäre die privacy cost im worst case unendlich).

Part 1: synthetic Data - wir erstellen ein Histogramm mit einer Bin pro Wert, für jede Bin wird ein Count-Query ausgeführt. durch parallel computing ist der privacy cost epsilon.

Der Range Count ist dann die Summe der noisy counts der Bins, die in der entsprechenden Range liegen.

Part 2: Wir bleiben beim Histogramm. Wir legen zusätzlich einen Threshold fest, unter dem der Fehler (mit Noise) bleiben muss, damit die synthetische Antwort ausgegeben werden kann.

Der Fehler ist `| RangeQuery(a,b) - SummeBins(a,b) |`. Um DP-konform zu bleiben verwendet SVT den verrauschten Fehler noisyError = `| RangeQuery(a,b) + Rauschen) - SummeBins(a,b) |`.

Wenn nun die Ungleichung noisyError >= (Threshold + Rauschen) wahr ist, wird eine Range Query auf die echten Daten ausgeführt. Im Zuge dessen werden auch die betroffenen Histogramm-Bins den echten Daten angepasst.

Somit entstehen extra privacy costs nur wenn die synthetischen Daten zu weit von dem eigentlichen Wert entfernt sind. Und da die synthetischen Daten danach angepasst werden, passiert das maximal einmal pro Query. Das Abfragen des RangeQuery bei der Berechnung des Fehlers fordert keine extra privacy cost, da das Ergebnis intern bleibt.

Part 3: Hier können wir die Strategie von Part 2 übernehmen.

## Checklist for Privacy Deployment

Deployment of differential privacy involves several steps to ensure the protection of sensitive data while still allowing useful analysis. Here's a checklist for deploying a system with differential privacy:

1. **Data Classification**: Identify and classify sensitive data that needs protection. Understand the sensitivity levels and potential risks associated with each class of data. This should include determination of the [Unit of Privacy](https://programming-dp.com/ch3.html#the-unit-of-privacy) as we previously discussed.

2. **Data Preparation**: Preprocess the data to remove personally identifiable information (PII) and other sensitive attributes that are not relevant to the analysis.

3. **Define Privacy Budget**: Determine acceptable levels of utility or privacy risk for your deployment. This will govern the amount of noise added to the data to achieve differential privacy.

4. **Data Analysis Requirements**: Clearly outline the data analysis goals, as well as specific transformations and queries that need to be supported while maintaining privacy. Ensure that the chosen privacy mechanisms and privacy unit can accommodate these requirements.

5. **Implement Differential Privacy**: Choose appropriate differential privacy mechanisms such as Laplace mechanism, Gaussian mechanism, or other advanced techniques based on the analysis requirements. Implement these mechanisms into the data processing pipeline. Identify a strategy for privacy composition if applicable.

6. **Noise Generation**: Generate and introduce high-quality entropy to the data in accordance with the chosen differential privacy mechanism. Ensure that the randomness level is calibrated to achieve the desired privacy guarantee.

7. **Testing and Verification**: Conduct thorough testing to validate the effectiveness of the deployed differential privacy mechanisms. Test the system with a variety of queries and scenarios to ensure that privacy is preserved while maintaining data utility. Ideally, conduct tests on public or synthetic data.

8. **Performance Evaluation**: Evaluate the performance overhead introduced by the time and storage cost of differential privacy mechanisms. Monitor system metrics such as latency, throughput, and compute resource utilization to ensure acceptable levels of efficiency.

9. **Documentation and Compliance**: Document the deployment process, including the differential privacy mechanisms used, privacy parameters chosen, and any other relevant details. Ensure compliance with relevant privacy laws, regulations and standards.

10. **Additional Security Measures**: Implement all necessary security measures to protect against potential attacks or vulnerabilities. This may include encryption of data in transit and at rest, access controls, and auditing mechanisms. This may also include safeguards against side channel attacks such as isolated compute environments.

11. **User Education and Training**: Educate users and stakeholders about the principles of differential privacy, its implications, and the importance of preserving privacy while conducting data analysis.

12. **Continuous Monitoring and Maintenance**: Establish a process for ongoing monitoring and maintenance of the deployed system. Regularly review privacy parameters and update mechanisms as needed to adapt to evolving privacy requirements and threats.

By following this checklist, you can effectively deploy a system with differential privacy to protect sensitive data while enabling valuable analysis and insights!